In [1]:
import pandas as pd
import re

In [2]:
#funciones 
def convertir_a_mayusculas(valor):
    if isinstance(valor, str):
        return valor.upper()
    else:
        return valor

def limpiar_y_completar(valor):
    """
    Elimina caracteres extraños y completa con ceros a la izquierda hasta 10 caracteres.
    :param valor: El valor que se desea procesar (string o número).
    :return: El valor limpio y completado como string.
    """
    # Convertir a string
    valor = str(valor)
    # Eliminar caracteres no numéricos usando expresiones regulares
    valor = re.sub(r'[^0-9]', '', valor)
    # Completar con ceros a la izquierda hasta 10 caracteres
    return valor.zfill(10)

# Ejemplo de uso
# print(limpiar_y_completar(' 0234-50abc '))  # Salida: '0000023450'
# print(limpiar_y_completar('123'))          # Salida: '0000000123'


def procesar_telefonos(telefono):
    telefono = str(telefono)
    # Reemplazar cualquier patrón de números separados por '-' por '\n'
    telefono = re.sub(r'(\d+)-(\d+)', r'\1\n\2', telefono)
    # Eliminar todos los guiones restantes
    telefono = telefono.replace('-', '')
    # Reemplazar dobles espacios por un salto de línea
    telefono = telefono.replace('  ', '\n')
    # Eliminar los espacios al inicio y al final
    telefono = telefono.strip()
    return telefono

def correc_mpio(valor):

    match valor:
        case 'CAPITAL - COBIJA':
            valor = 'COBIJA'
        case 'CAPITAL - LA PAZ':
            valor = 'NUESTRA SEÑORA DE LA PAZ'
        case 'CAPITAL - ORURO':
            valor = 'ORURO'
        case 'CAPITAL - POTOSÍ':
            valor = 'POTOSI'
        case 'CAPITAL - SANTA CRUZ DE LA SIERRA':
            valor = 'SANTA CRUZ DE LA SIERRA'
        case 'CAPITAL - SUCRE':
            valor = 'SUCRE'
        case 'CAPITAL - TRINIDAD':
            valor = 'TRINIDAD'
        case 'GUAYARAMERÍN':
            valor = 'GUAYARAMERIN'
        case 'YAPACANÍ':
            valor = 'YAPACANI'
        case 'CONCEPCIÓN':
            valor = 'CONCEPCION'
    
    return valor

In [3]:
# Carga del archivo principal
df = pd.read_csv('data.csv',sep=";", encoding="Latin")

# selección de columnas para carga de datos
df1 = df.copy()
df1 = df1[['CODIGO REGISTRO', 'FECHA REGISTRO','NOMBRE DE LA UNIDAD', 'DEPARTAMENTO', 'MUNICIPIO', 'CAEB','CIRC']]

In [4]:
# CAMBIO A MAYÚSCULAS LOS DATOS PRINCIPALES
df1['DEPARTAMENTO'] = df1['DEPARTAMENTO'].apply(convertir_a_mayusculas)
df1['MUNICIPIO'] = df1['MUNICIPIO'].apply(convertir_a_mayusculas)

# CORRECCIÓN DE LOS DATOS DE CODIGO DE REGISTRO
df1['CODIGO REGISTRO'] = df1['CODIGO REGISTRO'].apply(limpiar_y_completar)
df1['MUNICIPIO'] = df1['MUNICIPIO'].apply(correc_mpio)


In [5]:
#CARGA DE LOS MUNICIPIOS DISPONIBLES EN LA BASE DE DATOS DE PRUEBA
#LOS MUNICIPIOS Y SU CÓDIGO HAN SIDO EXPORTADOS EN CSV Y LLEVADOS A UN DICCIONARIO
mpio = pd.read_csv('municipios.csv',encoding='utf8',dtype='str')
dic_municipios = dict(zip(mpio['concat'],mpio['mpio']))

In [6]:
data = df1.copy()
data['mpio_registrado'] = data['CODIGO REGISTRO'].str[:6]
data['mpio_base'] = data['mpio_registrado'].apply(lambda x: dic_municipios[x] if x in dic_municipios else 'CÓDIGO NO ENCONTRADO')
data['cod_validate'] = data['mpio_registrado'].apply(lambda x: True if x in dic_municipios else False)
data['mpio_validate'] = data['MUNICIPIO'] == data['mpio_base']
data['control_final'] = data['cod_validate'] & data['mpio_validate']
data

,CODIGO REGISTRO,FECHA REGISTRO,NOMBRE DE LA UNIDAD,DEPARTAMENTO,MUNICIPIO,CAEB,CIRC,mpio_registrado,mpio_base,cod_validate,mpio_validate,control_final
0,0201040615,2020-11-23,PANADERIA FLORES,LA PAZ,EL ALTO,15411,4,020104,ACHOCALLA,True,False,False
1,0501010201,2017-12-04,CIP HILAAATURAA DE POTOSI,POTOSI,POTOSI,17112\n17111,4,050101,POTOSI,True,True,True
2,0601010468,2015-02-18,ASERRADERO - CARPINTERIA EL TALANO,TARIJA,CERCADO,36101\n20100,4,060101,TARIJA,True,False,False
3,0201001552,2019-02-25,PASTENPLAST,LA PAZ,PALCA,25201,4,020100,CÓDIGO NO ENCONTRADO,False,False,False
4,0201043451,2017-06-29,"METAL MECÁNICA ""TRIGUERO II""",LA PAZ,EL ALTO,28110,4,020104,ACHOCALLA,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
11652,0000002812,2014-02-26,CPAL SRL,LA PAZ,EL ALTO,15201\n15203,4,000000,CÓDIGO NO ENCONTRADO,False,False,False
11653,0000000835,2010-11-21,TRITURADORA Y SELECCIONADORA DE ARIDOS VAFERCON,BENI,TRINIDAD,26990\n37200,3,000000,CÓDIGO NO ENCONTRADO,False,False,False
11654,0000001202,2010-12-31,CHOCOLATES BREICK,LA PAZ,NUESTRA SEÑORA DE LA PAZ,15432,3,000000,CÓDIGO NO ENCONTRADO,False,False,False
11655,0000000711,2013-03-22,INDUSTRIAS ALIMENTICIAS ALENKA SRL,LA PAZ,EL ALTO,15134,4,000000,CÓDIGO NO ENCONTRADO,False,False,False


In [7]:
# Establecer la precisión a 2 decimales
pd.set_option('display.float_format', '{:.2f}'.format)
# Calcular los porcentajes
porcentajes = data['control_final'].value_counts(normalize=True) * 100

# Convertir a un DataFrame
tabla_porcentajes = porcentajes.reset_index()
tabla_porcentajes.columns = ['Valor', 'Porcentaje']

print('Datos errados, se consideró la equivalencia de código de mpio y mpio: ')
print(tabla_porcentajes)

Datos errados, se consideró la equivalencia de código de mpio y mpio: 
   Valor  Porcentaje
0  False       63.45
1   True       36.55


In [8]:
data.to_excel('Analisis.xlsx')

In [9]:
data.CIRC.unique()

array(['4', '3', '1 y 2', 'CUATRO (4)', '1 Y 2', 'TRES (3)', nan,
       'TRES(3)', 'CUATRO(4)', '1-2', '04', '1', '1Y2', '1y2', '2',
       '2021', '1 y2', '0', '5', '2018', 'CAT 1 Y 2', '2023', 'TRES (3)º',
       '123', '2020', '1,2', '-', '36101', '1 2'], dtype=object)

In [39]:
def corr_circ(valor):
    cat_12 = ['1 y 2','1 Y 2','1-2', '1', '1Y2', '1y2', '2','1 y2', 'CAT 1 Y 2','1,2', '1 2']
    cat_3 = ['3','TRES (3)','TRES(3)','TRES (3)º']
    cat_4 = ['4','CUATRO (4)','CUATRO(4)', '04']

    if valor in cat_12:
        return '2'
    elif valor in cat_3:
        return '3'
    elif valor in cat_4:
        return '4'
    else:
        return '1'

In [40]:
data['CIRC'] = data['CIRC'].apply(corr_circ)
data.CIRC.unique()

array(['4', '3', '2', '1'], dtype=object)

In [41]:
data_expanded = (
    data
    .assign(CAEB_SEP=data['CAEB'].str.split('\n'))  # Dividir por '\n'
    .explode('CAEB_SEP')  # Expandir los registros
    .reset_index(drop=True)  # Resetear los índices
)

print(data_expanded)

      CODIGO REGISTRO FECHA REGISTRO  \
0          0201040615     2020-11-23   
1          0501010201     2017-12-04   
2          0501010201     2017-12-04   
3          0601010468     2015-02-18   
4          0601010468     2015-02-18   
...               ...            ...   
14658      0000001202     2010-12-31   
14659      0000000711     2013-03-22   
14660      0000001728     2011-11-24   
14661      0000001728     2011-11-24   
14662      0000001728     2011-11-24   

                                     NOMBRE DE LA UNIDAD DEPARTAMENTO  \
0                                      PANADERIA  FLORES       LA PAZ   
1                              CIP HILAAATURAA DE POTOSI       POTOSI   
2                              CIP HILAAATURAA DE POTOSI       POTOSI   
3                     ASERRADERO - CARPINTERIA EL TALANO       TARIJA   
4                     ASERRADERO - CARPINTERIA EL TALANO       TARIJA   
...                                                  ...          ...   
14658   

In [42]:
data_expanded.to_excel('Lista_expandida.xlsx')

In [43]:
data_expanded.columns

Index(['CODIGO REGISTRO', 'FECHA REGISTRO', 'NOMBRE DE LA UNIDAD',
       'DEPARTAMENTO', 'MUNICIPIO', 'CAEB', 'CIRC', 'mpio_registrado',
       'mpio_base', 'cod_validate', 'mpio_validate', 'control_final',
       'CAEB_SEP'],
      dtype='object')

In [44]:
data_save = data_expanded[['DEPARTAMENTO', 'MUNICIPIO','CODIGO REGISTRO','FECHA REGISTRO','CAEB_SEP', 'CIRC','NOMBRE DE LA UNIDAD']]
data_save = data_save.sort_values(by='FECHA REGISTRO').reset_index(drop=True)
data_save.to_csv('carga.csv',encoding='utf8')

In [45]:
data.CIRC.unique()

array(['4', '3', '2', '1'], dtype=object)